In [ ]:
# sorry for installs here, I'll move them to poetry later
! pip install -U jupyter ipywidgets # for tqdm to function properly
! pip install openpyxl # for saving stuff to Excel files

In [ ]:
# import sys
import json
import uuid
from tqdm.notebook import tqdm
import datetime 
from openpyxl import Workbook

# path is broken on my machine, so I leave this here for myself :)
# sys.path.append('/Users/veronicasmilga/Desktop/Tübingen/MSE/Project_MSE/')

from db.DocumentEntry import DocumentEntry
from db.DocumentRepository import DocumentRepository
from data_retrieval.Crawler import Crawler

In [5]:
# frontier now is in a separate file
with open("../frontier.json", "r") as file:
    frontier = json.load(file)

NB: for the database to function properly, please first go to `exp/001_Flat_db_example_connection.ipynb` and complete the steps from there. If you don't want to be saving documents to the database, just comment out the code after _"# save one crawled page to database"_ comment.

In [ ]:
# initialising the database
documentRepository = DocumentRepository()

# initialising the Excel backup (if sth goes wrong with the database)
wb = Workbook()
ws = wb.active
ws.title = "Crawled Data"
headers = ["id", "url", "title", "headings", "page_text", "keywords", "accessed_timestamp", "internal_links", "external_links"]
ws.append(headers)

In [7]:
now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

max_pages = 1
max_steps_per_domain = 10
timeout = 10

# Crawler is an iterator now, to handle info generated on-the-fly and save it immediately
crawler = Crawler(frontier, max_pages, max_steps_per_domain, timeout)

for scraped_webpage_info in tqdm(crawler, total=max_pages):
    # save one crawled page to excel file
    row = [
        str(uuid.uuid4()),
        scraped_webpage_info["url"],
        scraped_webpage_info["title"],
        str(scraped_webpage_info["headings"]),
        scraped_webpage_info["page_text"],
        str(scraped_webpage_info["keywords"]),
        scraped_webpage_info["accessed_timestamp"],
        str(scraped_webpage_info["internal_links"]),
        str(scraped_webpage_info["external_links"])
    ]
    ws.append(row)
    wb.save(f"./data/crawled_data_backup_{now}.xlsx")

    # save one crawled page to database
    document = DocumentEntry(
        url=scraped_webpage_info["url"],
        title=scraped_webpage_info["title"],
        headings=scraped_webpage_info["headings"],
        page_text=scraped_webpage_info["page_text"], 
        keywords=scraped_webpage_info["keywords"],
        accessed_timestamp=scraped_webpage_info["accessed_timestamp"],
        internal_links=scraped_webpage_info["internal_links"],
        external_links=scraped_webpage_info["external_links"],
        id=uuid.uuid4()
        )
    documentRepository.saveDocument(document)

  0%|          | 0/1 [00:00<?, ?it/s]

Pages crawled: 0. Pages left: 1.
Crawling: https://uni-tuebingen.de/en/
Indexing...
Indexed url https://uni-tuebingen.de/en/ with title `Home | University of Tübingen` successfully.
Filtered out invalid internal link: tel:+4970712974444
Filtered out invalid internal link: javascript:linkTo_UnCryptMailto(%27ocknvq%2CuvwfkgtgpfgpugmtgvctkcvBbx0wpk%5C%2Fvwgdkpigp0fg%27);
Filtered out invalid internal link: tel:+497071290


If you want to access documents in the database:

In [ ]:
allDocuments = documentRepository.loadAllDocuments()

SC: Deleted all documents.


[]

Otherwise, if you just want to take a look at the data, go to `exp/data` and find the Excel file with timestamp corresponding to the time you ran the Crawler :)